# Purchase-based Recommendation System
## Based on Moorissa Tjokro tutorial
### Coded by Rebeca Bivar - DB: Armazem Paraíba

### Imports and reading file (data here has only clients, products and purchase count)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import turicreate as tc
from sklearn.model_selection import train_test_split


In [2]:
#READING A FILE WITH ONLY CLIENTS, PRODUCTS BOUGHT AND QTDE
buyers = pd.read_csv('purchase_limpa.csv', sep=';')
buyers.head()

,Unnamed: 0,COD_CLIENTE,COD_PRODUTO,NOME_PRODUTO,QUANTIDADE,CANAL
0,0,05190001,25172,SMARTPHONE LG K9 TV LM-X210BMW PRETO,1,VENDAWEB
1,1,37578201,25367,TABLET NB729 MINI MS40G BRANCO,1,VENDAWEB
2,2,93168801,25354,"TELEVISOR SMART 32"" UN32J4290 SAMSUNG",1,VENDAWEB
3,3,62982901,26584,SMARTPHONE MOTOROLA MOTO G8 PLUS 64GB CEREJA,1,VENDAWEB
4,4,54509401,21647,DVD D-15 KARAOKE MONDIAL,1,VENDAWEB


##  Data preparation
### Creating dummy table to check if the client has bought a product or not

In [3]:
def create_data_dummy(db):
    data_dummy = db.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy

data_dummy = create_data_dummy(buyers)
data_dummy


,Unnamed: 0,COD_CLIENTE,COD_PRODUTO,NOME_PRODUTO,QUANTIDADE,CANAL,purchase_dummy
0,0,05190001,25172,SMARTPHONE LG K9 TV LM-X210BMW PRETO,1,VENDAWEB,1
1,1,37578201,25367,TABLET NB729 MINI MS40G BRANCO,1,VENDAWEB,1
2,2,93168801,25354,"TELEVISOR SMART 32"" UN32J4290 SAMSUNG",1,VENDAWEB,1
3,3,62982901,26584,SMARTPHONE MOTOROLA MOTO G8 PLUS 64GB CEREJA,1,VENDAWEB,1
4,4,54509401,21647,DVD D-15 KARAOKE MONDIAL,1,VENDAWEB,1
...,...,...,...,...,...,...,...
291400,291400,9OI04701,24716,RECEPTOR MIDIA BOX HDTV B3,1,VENDAWEB,1
291401,291401,99FZPR01,25981,SMARTPHONE SAMSUNG GAL A10 A105 32GB PRETO,1,VENDAWEB,1
291402,291402,9NYVEB01,25981,SMARTPHONE SAMSUNG GAL A10 A105 32GB PRETO,1,VENDAWEB,1
291403,291403,99K0N601,25727,SMARTPHONE SAMSUNG GAL J2 CORE 16GB PRATA,1,VENDAWEB,1


### Normalizing purchase frequency of each item across users 

In [4]:
#Dummy for marking whether a customer bought that item or not

df_matrix = pd.pivot_table(buyers, values = 'QUANTIDADE', index = 'COD_CLIENTE', columns = 'COD_PRODUTO')


df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

# create a table for input to the modeling  
data_input = df_matrix_norm.reset_index()
data_input.index.names = ['FREQ_COMPRAS']
data_norm = pd.melt(data_input, id_vars=['COD_CLIENTE'], 
                    value_name='FREQ_COMPRAS')

#print(data_norm.shape)
#data_norm.head()

In [5]:
#Just cleaning useless values 
data_norm = data_norm.dropna()
data_norm

,COD_CLIENTE,COD_PRODUTO,FREQ_COMPRAS
1085593,00017101,11332,0.0
1088205,00449301,11332,0.0
1088477,00846001,11332,0.0
1089371,02130901,11332,0.0
1090227,03152901,11332,0.0
...,...,...,...
152631320,YZ0WFN01,26635,0.0
152631324,YZ0WH901,26635,0.0
152631337,YZ0WM301,26635,0.0
152631435,YZ0XPA01,26635,0.0


### Split data into trainning and testing (80/20)

In [6]:

# Returns train and test datasets as scalable dfs
def split_data(data):
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

# Now actually splitting purchase_counts, purchase_dummy and purchase_counts_norm
train_data, test_data = split_data(buyers)
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

## Baseline model to compare and evaluate models


In [7]:
# variables to define field names: 

user_id = 'COD_CLIENTE'
item_id = 'COD_PRODUTO'
item_name = 'NOME_PRODUTO'
users_to_recommend = list(buyers[user_id])
n_recommendation = 10 # itens to recommend
n_display = 30 # display the first few rows in an output dataset

# Function for all models using turicreate
def model(train_data, name, user_id, item_id, target, 
          users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id = user_id, 
                                                    item_id = item_id,
                                                    target = target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

### Popularity Model
   Takes the most popular items for recommendation, which are the products with the highest number of sells across customers.

In [18]:
name = 'popularity'
target = 'QUANTIDADE'
popularity, recom_pop = model(train_data, name, user_id, item_id, target, 
                   users_to_recommend, n_recommendation, n_display)
popularity, recom_pop

Warning: Ignoring columns Unnamed: 0, NOME_PRODUTO, CANAL;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 233124 observations with 182327 users and 675 items.

Data prepared in: 0.186049s

...

recommendations finished on 287000/291405 queries. users per second: 519015

recommendations finished on 288000/291405 queries. users per second: 518694

recommendations finished on 289000/291405 queries. users per second: 517272

recommendations finished on 290000/291405 queries. users per second: 516293

recommendations finished on 291000/291405 queries. users per second: 515158

In [9]:
name = 'popularity'
target = 'purchase_dummy'
pop_dummy, recom_pop_dummy = model(train_data_dummy, name, user_id, item_id, target, 
                   users_to_recommend, n_recommendation, n_display)

Warning: Ignoring columns Unnamed: 0, NOME_PRODUTO, QUANTIDADE, CANAL;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 233124 observations with 182493 users and 669 items.

Data prepared in: 0.170639s

...

recommendations finished on 287000/291405 queries. users per second: 532920

recommendations finished on 288000/291405 queries. users per second: 532196

recommendations finished on 289000/291405 queries. users per second: 531458

recommendations finished on 290000/291405 queries. users per second: 530743

recommendations finished on 291000/291405 queries. users per second: 528368

### Using scaled purchase count 

In [10]:
name = 'popularity'
target = 'FREQ_COMPRAS'
pop_norm, recom_pop_norm = model(train_data_norm, name, user_id, item_id, target, 
                 users_to_recommend, n_recommendation, n_display)

Preparing data set.

Data has 213236 observations with 173290 users and 245 items.

Data prepared in: 0.1564s

213236 observations to process; with 245 unique items.

recommendations finished on 1000/291405 queries. users per second: 138715

...

recommendations finished on 287000/291405 queries. users per second: 595631

recommendations finished on 288000/291405 queries. users per second: 595084

recommendations finished on 289000/291405 queries. users per second: 594395

recommendations finished on 290000/291405 queries. users per second: 593838

recommendations finished on 291000/291405 queries. users per second: 593501

### Collaborative Filtering Model

   Recommends items based on how similar clients purchase items. Meaning: if customer 1 and customer 2 bought similar items, for example, 1 bought X, Y, Z and 2 bought X, Y, we would recommend an item Z to customer 2.
    
   - Lets say X and Y have been rated by costumers 1 and 2. 
   - We then create two item-vectors for both items, then we find the **cosine** or **pearson** distance between these vectors. If the **cosine** value is 1, means total similarity, if it is 0, means no similarity.
   - In this case, we will check the similarity between the target item and other items the customer already bought - using the client's purchase count to items already bought by him as weighing factor (some sort of simulated rating). 

### Using purchase count and purchase frequency
### Cosine

In [11]:
name = 'cosine'
target = 'QUANTIDADE'
cos, cos_recom = model(train_data, name, user_id, item_id, target, 
            users_to_recommend, n_recommendation, n_display)

Warning: Ignoring columns Unnamed: 0, NOME_PRODUTO, CANAL;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 233124 observations with 182327 users and 675 items.

Data prepared in: 0.171566s

...

recommendations finished on 287000/291405 queries. users per second: 398653

recommendations finished on 288000/291405 queries. users per second: 396972

recommendations finished on 289000/291405 queries. users per second: 395481

recommendations finished on 290000/291405 queries. users per second: 393551

recommendations finished on 291000/291405 queries. users per second: 388842

In [12]:
name = 'cosine'
target = 'purchase_dummy'
cos_dummy, cos_recom_dummy = model(train_data_dummy, name, user_id, item_id, target,
                  users_to_recommend, n_recommendation, n_display)

Warning: Ignoring columns Unnamed: 0, NOME_PRODUTO, QUANTIDADE, CANAL;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 233124 observations with 182493 users and 669 items.

Data prepared in: 0.157041s

...

recommendations finished on 287000/291405 queries. users per second: 424901

recommendations finished on 288000/291405 queries. users per second: 422780

recommendations finished on 289000/291405 queries. users per second: 419336

recommendations finished on 290000/291405 queries. users per second: 416224

recommendations finished on 291000/291405 queries. users per second: 411472

In [13]:
name = 'cosine'
target = 'FREQ_COMPRAS'
cos_norm, cos_recom_norm = model(train_data_norm, name, user_id, item_id, target, 
                   users_to_recommend, n_recommendation, n_display)


Preparing data set.

Data has 213236 observations with 173290 users and 245 items.

Data prepared in: 0.143522s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 287000/291405 queries. users per second: 500458

recommendations finished on 288000/291405 queries. users per second: 499440

recommendations finished on 289000/291405 queries. users per second: 497161

recommendations finished on 290000/291405 queries. users per second: 494605

recommendations finished on 291000/291405 queries. users per second: 492244

### Using purchase count and purchase frequency
### Pearson

In [14]:
# PURCHASE COUNT
name = 'pearson'
target = 'QUANTIDADE'
pear, pear_recom = model(train_data, name, user_id, item_id, target,
             users_to_recommend, n_recommendation, n_display)

Warning: Ignoring columns Unnamed: 0, NOME_PRODUTO, CANAL;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 233124 observations with 182327 users and 675 items.

Data prepared in: 0.169034s

...

recommendations finished on 288000/291405 queries. users per second: 308231

recommendations finished on 289000/291405 queries. users per second: 306618

recommendations finished on 290000/291405 queries. users per second: 303843

recommendations finished on 291000/291405 queries. users per second: 301289

+-------------+-------------+--------------------+------+
| COD_CLIENTE | COD_PRODUTO |       score        | rank |
+-------------+-------------+--------------------+------+
|   05190001  |    20249    |        5.0         |  1   |
|   05190001  |    16232    |        4.0         |  2   |
|   05190001  |    14314    |        2.0         |  3   |
|   05190001  |    20250    |        2.0         |  4   |
|   05190001  |    18662    |        2.0         |  5   |
|   05190001  |    17372    |        2.0         |  6   |
|   05190001  |    17196    |        2.0         |  7   |
|   05190001  |    24598    | 1.4818840579710144 |  8   |
|   05190001  |    24597    | 1.338709677419355  |  9   |
|   05190001  |    23193    | 1.3333333333333333 |  10  |
|   37578201  |    20249    |        5.0         |  1   |
|   37578201  |    16232    |        4.0         |  2   |
|   37578201  |    14314    |        2.0         |  3   |
|   37578201  |    20250    |        2.0         |  4   |
|   37578201  

In [15]:
# PURCHASE DUMMY
name = 'pearson'
target = 'purchase_dummy'
pear_dummy, pear_recom_dummy = model(train_data_dummy, name, user_id, item_id, target,
                   users_to_recommend, n_recommendation, n_display)

Warning: Ignoring columns Unnamed: 0, NOME_PRODUTO, QUANTIDADE, CANAL;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 233124 observations with 182493 users and 669 items.

Data prepared in: 0.173133s

...

recommendations finished on 288000/291405 queries. users per second: 385499

recommendations finished on 289000/291405 queries. users per second: 382029

recommendations finished on 290000/291405 queries. users per second: 379148

recommendations finished on 291000/291405 queries. users per second: 376321

+-------------+-------------+-------+------+
| COD_CLIENTE | COD_PRODUTO | score | rank |
+-------------+-------------+-------+------+
|   05190001  |    25241    |  0.0  |  1   |
|   05190001  |    25981    |  0.0  |  2   |
|   05190001  |    21653    |  0.0  |  3   |
|   05190001  |    26631    |  0.0  |  4   |
|   05190001  |    26423    |  0.0  |  5   |
|   05190001  |    26006    |  0.0  |  6   |
|   05190001  |    26005    |  0.0  |  7   |
|   05190001  |    19525    |  0.0  |  8   |
|   05190001  |    19729    |  0.0  |  9   |
|   05190001  |    25173    |  0.0  |  10  |
|   37578201  |    25241    |  0.0  |  1   |
|   37578201  |    25981    |  0.0  |  2   |
|   37578201  |    21653    |  0.0  |  3   |
|   37578201  |    26631    |  0.0  |  4   |
|   37578201  |    26423    |  0.0  |  5   |
|   37578201  |    26006    |  0.0  |  6   |
|   37578201  |    26005    |  0.0  |  7   |
|   37578201  |    19525    |  0.0  |  8   |
|   37578201  |    19729    |  0.0  |  9   |
|   375782

In [16]:
# PURCHASE FREQUENCY
name = 'pearson'
target = 'FREQ_COMPRAS'
pear_norm, pear_recom_norm = model(train_data_norm, name, user_id, item_id, target,
                  users_to_recommend, n_recommendation, n_display)

Preparing data set.

Data has 213236 observations with 173290 users and 245 items.

Data prepared in: 0.156834s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 287000/291405 queries. users per second: 387173

recommendations finished on 288000/291405 queries. users per second: 385508

recommendations finished on 289000/291405 queries. users per second: 383487

recommendations finished on 290000/291405 queries. users per second: 381611

recommendations finished on 291000/291405 queries. users per second: 380180

## Model Evaluation 

### RMSE - Root Mean Squared Errors

   - Measures the error of predicted values
   - Lesser the RMSE values, better the recommendations
   
### Precision-Recall

   - Recall: Percentage of products that a customer buys that are actually recommended. 
   - Precision: How many itens the customer liked out of the recommended?
   - The idea is to optimze both recall and precision to be close as 1 as possible


In [19]:
# Variables for model evaluation

models_counts = [popularity, cos, pear]
models_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_norm = [pop_norm, cos_norm, pear_norm]

names_counts = ['Popularity Model on Purchase Counts', 
                  'Cosine Similarity on Purchase Counts', 
                  'Pearson Similarity on Purchase Counts']
names_dummy = ['Popularity Model on Purchase Dummy', 
                 'Cosine Similarity on Purchase Dummy', 
                 'Pearson Similarity on Purchase Dummy']
names_norm = ['Popularity Model on Scaled Purchase Counts', 
                'Cosine Similarity on Scaled Purchase Counts', 
                'Pearson Similarity on Scaled Purchase Counts']


eval_counts = tc.recommender.util.compare_models(test_data, models_counts, 
                                                 model_names=names_counts)
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_dummy,
                                                model_names=names_dummy)
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_norm, 
                                               model_names=names_norm)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/54086 queries. users per second: 100492

recommendations finished on 2000/54086 queries. users per second: 139811

recommendations finished on 3000/54086 queries. users per second: 159549

recommendations finished on 4000/54086 queries. users per second: 175032

...

recommendations finished on 47000/50209 queries. users per second: 124784

recommendations finished on 48000/50209 queries. users per second: 123685

recommendations finished on 49000/50209 queries. users per second: 122975

recommendations finished on 50000/50209 queries. users per second: 121593


Precision and recall summary statistics by cutoff
+--------+-----------------------+------------------------+
| cutoff |     mean_precision    |      mean_recall       |
+--------+-----------------------+------------------------+
|   1    |          0.0          |          0.0           |
|   2    |          0.0          |          0.0           |
|   3    |          0.0          |          0.0           |
|   4    | 5.477105698181566e-05 | 0.00018920910593718204 |
|   5    | 5.975024398016292e-05 | 0.00026887609791073397 |
|   6    | 5.643078598126479e-05 | 0.0003087095938975075  |
|   7    | 5.690499426682184e-05 | 0.0003684598378776719  |
|   8    | 4.979186998346907e-05 | 0.0003684598378776714  |
|   9    | 4.647241198457114e-05 | 0.00037841821187436635 |
|   10   | 4.381684558545269e-05 | 0.00039833495986775307 |
+--------+-----------------------+------------------------+
[10 rows x 3 columns]


Overall RMSE: 0.05246546764609195

Per User RMSE (best)
+-------------+------+-------

## Final Model

### Cosine Similarity - Purchase Dummy

In [29]:
final_model = tc.item_similarity_recommender.create(tc.SFrame(data_dummy),
                                                   user_id = user_id,
                                                   item_id = item_id,
                                                   target = 'purchase_dummy',
                                                   similarity_type='cosine')
recom = final_model.recommend(users=users_to_recommend, k=10)
recom.print_rows(n_display)

Warning: Ignoring columns Unnamed: 0, NOME_PRODUTO, QUANTIDADE, CANAL;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 291405 observations with 217115 users and 710 items.

Data prepared in: 0.171117s

...

recommendations finished on 287000/291405 queries. users per second: 459532

recommendations finished on 288000/291405 queries. users per second: 457245

recommendations finished on 289000/291405 queries. users per second: 455210

recommendations finished on 290000/291405 queries. users per second: 454145

recommendations finished on 291000/291405 queries. users per second: 450757

In [30]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()

(2914050, 4)


,COD_CLIENTE,COD_PRODUTO,score,rank
0,05190001,26245,0.110144,1
1,05190001,25481,0.050211,2
2,05190001,25173,0.047190,3
3,05190001,24089,0.016027,4
4,05190001,24316,0.016027,5


## Output Dataframe

    Contains all recommendations

In [38]:
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['COD_CLIENTE', 'recommendedProducts']].drop_duplicates() \
        .sort_values('COD_CLIENTE').set_index('COD_CLIENTE')
    if print_csv:
        df_output.to_csv('../output/option1_recommendation.csv', sep=';')
        print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")
    return df_output

In [37]:
df_output = create_output(final_model, users_to_recommend, 10, print_csv=True)
print(df_output.shape)
df_output.head()

recommendations finished on 1000/291405 queries. users per second: 196309

recommendations finished on 2000/291405 queries. users per second: 305297

recommendations finished on 3000/291405 queries. users per second: 370737

recommendations finished on 4000/291405 queries. users per second: 422476

recommendations finished on 5000/291405 queries. users per second: 428192

...

recommendations finished on 289000/291405 queries. users per second: 451831

recommendations finished on 290000/291405 queries. users per second: 450659

recommendations finished on 291000/291405 queries. users per second: 448726

An output file can be found in 'output' folder with name 'option1_recommendation.csv'
(217115, 1)


,recommendedProducts
COD_CLIENTE,
00000301,15558|14537|19729|20966|24654|21823|21821|2524...
00001501,25727|25729|25728|25983|25981|25982|25172|2165...
00002701,26243|21653|24716|25727|25546|25726|25997|1780...
00003101,25726|21256|25729|25982|25983|25728|21854|2548...
00004001,25982|25983|25729|25727|25726|26003|25728|2535...


## Customer Recommendation

### Given a 'Client Code', shows all suggested products

In [41]:
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]

In [45]:
a = customer_recomendation('00003101')

In [46]:
a

recommendedProducts    25726|21256|25729|25982|25983|25728|21854|2548...
Name: 00003101, dtype: object